In [1]:
from components.HarvestWorld import HarvestWorld

from importlib import import_module, reload

import ipywidgets as widgets

import matplotlib.pyplot as plt
from IPython.display import clear_output

harvest_world_module = import_module('components.HarvestWorld')
reload(harvest_world_module)

<module 'components.HarvestWorld' from '/home/raul/Escritorio/extra/misis/rl/rl_uniandes/dynamic_actions/components/HarvestWorld.py'>

In [2]:
ASCII_MAP = """
A     A
AA   AA
A  Q  A
"""

harvest_world = HarvestWorld(ASCII_MAP, 
                            regrowth_probs=[0.0, 0.005],
                            num_agents=1)
    

In [3]:
def update_plot(freq):
    clear_output(wait=True)
    fig, axes = plt.subplots(1, 2, figsize=(20, 15))
    harvest_world.plot_map(axes[0])
    harvest_world.agents[0].plot_knowledge(axes[1])
    harvest_world.advance_timestep()
    plt.show() 
    
freq_slider = widgets.FloatSlider(value=0, min=0, max=10, step=1, description='Frequency:')
out = widgets.interactive_output(update_plot, {'freq': freq_slider})
display(freq_slider, out)

FloatSlider(value=0.0, description='Frequency:', max=10.0, step=1.0)

Output()

In [4]:
harvest_world.agents[0].q_knowledge[(1,1)]

{}

In [5]:
harvest_world.agents[0].q_knowledge[(1,0)]

{}

In [6]:
harvest_world.agents[0].current_state

(2, 4)

In [7]:
import numpy as np

In [8]:
q_table = np.random.uniform(low=0, high=1, size= (2,7,4))
q_table

array([[[0.72592344, 0.14557076, 0.5961265 , 0.47771621],
        [0.55860462, 0.88805247, 0.57302768, 0.47344708],
        [0.85934897, 0.74580315, 0.03963492, 0.15347574],
        [0.4855246 , 0.06988748, 0.01717942, 0.41553516],
        [0.01203677, 0.20517753, 0.30600817, 0.81399538],
        [0.3841488 , 0.50492942, 0.63375073, 0.4886411 ],
        [0.0214846 , 0.92005486, 0.32362265, 0.59537093]],

       [[0.8176766 , 0.10539605, 0.22780485, 0.45755548],
        [0.08871188, 0.33832338, 0.43059524, 0.32728327],
        [0.40430223, 0.85156506, 0.67953554, 0.82261838],
        [0.47252512, 0.18244309, 0.3281988 , 0.18852174],
        [0.90666567, 0.69476047, 0.45938165, 0.17333131],
        [0.61218789, 0.17815319, 0.58754852, 0.6575252 ],
        [0.8621788 , 0.7443834 , 0.30549908, 0.74695082]]])

In [9]:
q_table.shape

(2, 7, 4)

In [10]:
np.argmax(q_table[(0,0)])

0

In [11]:
q_table[(0,0)]

array([0.72592344, 0.14557076, 0.5961265 , 0.47771621])

In [12]:
q_table[(1,2)]

array([0.40430223, 0.85156506, 0.67953554, 0.82261838])

In [13]:
np.max(q_table[(0,0)])

0.7259234367957242